# Linked Lists

## Agenda

1. The `LinkedList` and `Node` classes  
2. Implementing `append`
3. Implementing deletion
4. Bidirectional links
5. Run-time analysis
6. Closing remarks

## 1. The `LinkedList` and `Node` classes

In [ ]:
class LinkedList:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next = next
    
    def __init__(self):
        self.head = None
        self.count = 0
    
    def prepend(self, value):
        pass
    
    def __len__(self):
        return self.count
        
    def __iter__(self):
        n = self.head
        while n:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(x for x in self) + ']'

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.prepend(i)
lst

## 2. Implementing `append`

### Option 1

In [ ]:
class LinkedList (LinkedList): # note: using inheritance to extend prior definition
    def append(self, value):
        pass

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

### Option 2

In [ ]:
class LinkedList (LinkedList):
    def __init__(self):
        self.head = self.tail = None
        self.count = 0
        
    def prepend(self, value):
        pass
        
    def append(self, value):
        pass

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

## 3. Implementing deletion

### Deleting the head

In [ ]:
class LinkedList (LinkedList):
    def del_head(self):
        assert(len(self) > 0)
        pass

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst.del_head()
lst.del_head()
lst

### Deleting the tail

In [ ]:
class LinkedList (LinkedList):
    def del_tail(self):
        assert(len(self) > 0)
        pass

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst.del_tail()
lst.del_tail()
lst

## 4. Bidirectional links (Doubly-linked list) & Sentinel head

In [ ]:
class LinkedList:
    class Node:
        def __init__(self, val, prior=None, next=None):
            self.val = val
            self.prior = prior
            self.next  = next
    
    def __init__(self):
        self.count = 0
        
    def prepend(self, value):
        self.count += 1
        
    def append(self, value):
        self.count += 1
        
    def __getitem__(self, idx):
        pass
        
    def __len__(self):
        return self.count
        
    def __iter__(self):
        n = self.head.next
        while n is not self.head:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.prepend(i)
for i in range(10):
    lst.append(i)
lst

## 5. Incorporating a "cursor"

In [ ]:
class LinkedList:
    class Node:
        def __init__(self, val, prior=None, next=None):
            self.val = val
            self.prior = prior
            self.next  = next
    
    def __init__(self):
        self.head = self.cursor = LinkedList.Node(None)
        self.head.prior = self.head.next = self.head
        self.count = 0
                
    def append(self, value):
        n = LinkedList.Node(value, prior=self.head.prior, next=self.head)
        n.prior.next = n.next.prior = n
        self.count += 1
        
    def cursor_set(self, idx):
        pass
    
    def cursor_insert(self, x):
        pass
    
    def cursor_delete(self):
        pass
        
    def __len__(self):
        return self.count
        
    def __iter__(self):
        n = self.head.next
        while n is not self.head:
            yield n.val
            n = n.next
    
    def __repr__(self):
        return '[' + ', '.join(str(x) for x in self) + ']'

In [ ]:
lst = LinkedList()
for i in range(10):
    lst.append(i)
lst

In [ ]:
lst.cursor_set(4)
for x in 'abcd':
    lst.cursor_insert(x)
lst

In [ ]:
lst.cursor_set(8)
for _ in range(4):
    lst.cursor_delete()

## 6. Run-time analysis

Run-time complexities for circular, doubly-linked list of $N$ elements:

- indexing (position-based access) = $O(?)$
- search (unsorted) = $O(?)$
- search (sorted) = $O(?)$ --- binary search isn't possible!
- prepend = $O(?)$
- append = $O(?)$
- indexing = $O(?)$
- insertion at arbitrary position: indexing = $O(?)$ + insertion = $O(?)$
- deletion of arbitrary element: indexing = $O(?)$ + deletion = $O(?)$